In [36]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [54]:
hh = pd.read_csv("/Users/ashrafzaman/matsim_pipelines/popgen_input_data/data_EOD_2018/od_hh.csv")
pp = pd.read_csv("/Users/ashrafzaman/matsim_pipelines/popgen_input_data/data_EOD_2018/od_per.csv")

In [58]:
hh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66406 entries, 0 to 73420
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   nolog       66406 non-null  int64  
 1   tlog        66406 non-null  float64
 2   nbper       66406 non-null  int64  
 3   nbveh       66406 non-null  int64  
 4   revenu      66406 non-null  int64  
 5   langue      66406 non-null  int64  
 6   nmodif_ass  66406 non-null  int64  
 7   incaplogi   66406 non-null  int64  
 8   xmtmlog     66406 non-null  int64  
 9   ymtmlog     66406 non-null  int64  
 10  r6log       66406 non-null  int64  
 11  r8log       66406 non-null  int64  
 12  rmrlog      66406 non-null  int64  
 13  arrlog      31782 non-null  object 
 14  sdrlog      66406 non-null  int64  
 15  srlog       66406 non-null  float64
 16  sdomi65     66406 non-null  int64  
 17  smlog       66406 non-null  int64  
 18  cplog       55397 non-null  object 
 19  agr_domi    66406 non-nul

## Filter data from the CMA of Montreal

In [55]:
hh['cma'] = [str(x)[:3] for x in hh.srlog]
hh = hh[hh['cma'] == '462'].copy()

In [56]:
pp = pp[pp.nolog.isin(hh.nolog.unique())]

### Check

In [57]:
assert len(pp.nolog.unique()) == len(hh)  # Check that every person is assigned to household

## Households

In [41]:
hh_with_kids = pp[pp['age'] < 19]['nolog'].unique()  # households with at least one child under 15 of age

In [42]:
hh_kids = hh.loc[hh['nolog'].isin(hh_with_kids)] # Do not include households that are solely formed by someone under 19y/o
hh_with_kids = hh_kids[hh_kids.nbper > 1].nolog.values

In [43]:
hh['hh_type'] = 2
hh.loc[hh['nolog'].isin(hh_with_kids), 'hh_type'] = 1

In [44]:
assert len(hh[hh['hh_type'] == 1]) == len(hh_with_kids)

In [45]:
hh_id_dict = dict(zip(pd.np.sort(list(hh.nolog)), range(len(hh))))

/var/folders/v6/djhtsx5d3198mkwrr88jb6tc0000gn/T/ipykernel_12544/3729937750.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  hh_id_dict = dict(zip(pd.np.sort(list(hh.nolog)), range(len(hh))))


In [46]:
hh['hid'] = [hh_id_dict[x] for x in hh.nolog]

## Geo matching

In [47]:
hh.rename(columns={'srlog': 'CTUID'}, inplace=True) # depending on the EOD (2013 or 2016), census tracts are not the same
hh['CTUID'] *= 1000

In [48]:
mapping_da_ct = gpd.read_file('output/geoFilter.shp')

In [49]:
mapping_da_ct 

,DAUID,CTUID,PRUID,geo,region,DGUID_DA,DGUID_CT,CTNAME,LANDAREA_D,LANDAREA_C,geometry
0,24520101,4620690.00,24,0,683,2021S051224520101,2021S05074620690.00,0690.00,3.9586,65.4346,"MULTIPOLYGON (((322843.003 5082101.000, 322841..."
1,24520110,4620690.00,24,1,683,2021S051224520110,2021S05074620690.00,0690.00,0.7806,65.4346,"POLYGON ((320942.854 5082073.208, 321265.310 5..."
2,24520111,4620690.00,24,2,683,2021S051224520111,2021S05074620690.00,0690.00,0.1939,65.4346,"POLYGON ((320860.655 5081730.179, 320858.791 5..."
3,24520112,4620690.00,24,3,683,2021S051224520112,2021S05074620690.00,0690.00,1.9319,65.4346,"MULTIPOLYGON (((321705.000 5081499.998, 321710..."
4,24520113,4620690.00,24,4,683,2021S051224520113,2021S05074620690.00,0690.00,20.8775,65.4346,"POLYGON ((320067.058 5082942.259, 320061.699 5..."
...,...,...,...,...,...,...,...,...,...,...,...
6571,24750179,4620792.00,24,6571,796,2021S051224750179,2021S05074620792.00,0792.00,4.3616,31.0979,"POLYGON ((258345.446 5072997.135, 258348.486 5..."
6572,24750180,4620792.00,24,6572,796,2021S051224750180,2021S05074620792.00,0792.00,9.4638,31.0979,"POLYGON ((258583.427 5076373.877, 258587.381 5..."
6573,24750181,4620792.00,24,6573,796,2021S051224750181,2021S05074620792.00,0792.00,6.5496,31.0979,"POLYGON ((261572.725 5074517.787, 261572.731 5..."
6574,24760050,4620740.01,24,6574,752,2021S051224760050,2021S05074620740.01,0740.01,24.6956,90.0416,"POLYGON ((250667.044 5067564.902, 250665.481 5..."


In [50]:
hh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66406 entries, 0 to 73420
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   nolog       66406 non-null  int64  
 1   tlog        66406 non-null  float64
 2   nbper       66406 non-null  int64  
 3   nbveh       66406 non-null  int64  
 4   revenu      66406 non-null  int64  
 5   langue      66406 non-null  int64  
 6   nmodif_ass  66406 non-null  int64  
 7   incaplogi   66406 non-null  int64  
 8   xmtmlog     66406 non-null  int64  
 9   ymtmlog     66406 non-null  int64  
 10  r6log       66406 non-null  int64  
 11  r8log       66406 non-null  int64  
 12  rmrlog      66406 non-null  int64  
 13  arrlog      31782 non-null  object 
 14  sdrlog      66406 non-null  int64  
 15  CTUID       66406 non-null  float64
 16  sdomi65     66406 non-null  int64  
 17  smlog       66406 non-null  int64  
 18  cplog       55397 non-null  object 
 19  agr_domi    66406 non-nul

In [51]:
ct_2016_ = pd.DataFrame(mapping_da_ct[['CTUID', 'region']].drop_duplicates())

In [52]:
ct_2016_

,CTUID,region
0,4620690.00,683
8,4620691.00,684
17,4620681.00,639
24,4620692.02,686
29,4620694.02,689
...,...,...
6553,4620790.01,792
6557,4620791.01,794
6563,4620780.00,780
6570,4620792.00,796


In [53]:
unmatched_ct_2016 = ct_2016_.loc[~ct_2016_.CTUID.isin(hh.ct_2011), "CTUID"]

AttributeError: 'DataFrame' object has no attribute 'ct_2011'

In [35]:
unmatched_ct_2016

375     4622003000
1028    4620887070
4006    4620315000
5093    4620094020
5806    4620440000
Name: CTUID, dtype: int64

In [36]:
mapping_ct_2011_2016 = pd.read_csv("../../../data/shp/mapping_ct_2011_ct_2016.csv")
mapping_ct_2011_2016 *= 1000

mapping_ct_2011_2016 = mapping_ct_2011_2016[["CTUID", "CTUID_2"]]

In [37]:
unmatched_ct_2016[~unmatched_ct_2016.isin(mapping_ct_2011_2016.CTUID)]

5093    4620094020
Name: CTUID, dtype: int64

In [683]:
ct_2016_.loc[~ct_2016_.CTUID.isin(unmatched_ct_2016), 'sample_geo'] = ct_2016_['new_ct']
ct_2016_.rename(columns={'CTUID': 'region'}, inplace=True)

In [712]:
test = ct_2016_[ct_2016_.sample_geo.isna()].merge(mapping_ct_2011_2016, left_on="region", right_on="CTUID").drop_duplicates()

In [720]:
start = test.new_ct.max() + 1
end = start + len(test.CTUID_2.unique())
ct_2011_ids = range(start, end)
ct_2011_ids_dict = dict(zip(test.CTUID_2.unique(), ct_2011_ids))

In [721]:
test['sample_geo'] = [ct_2011_ids_dict.get(x, None) for x in test.CTUID_2]

In [723]:
test[test.sample_geo.isna()]

,region,new_ct,sample_geo,CTUID,CTUID_2


In [745]:
reg_sample = pd.concat([ct_2016_[~ct_2016_.sample_geo.isna()], test], axis=0)

In [751]:
reg_sample.loc[reg_sample.CTUID.isna(), 'CTUID_2'] = reg_sample.loc[reg_sample.CTUID.isna(), 'region']

In [752]:
reg_sample

,region,new_ct,sample_geo,CTUID,CTUID_2,srlog
0,4620691000,658,658.0,NaN,4.620691e+09,4.620691e+09
9,4620882000,874,874.0,NaN,4.620882e+09,4.620882e+09
14,4620881020,873,873.0,NaN,4.620881e+09,4.620881e+09
21,4620881010,872,872.0,NaN,4.620881e+09,4.620881e+09
28,4620873010,857,857.0,NaN,4.620873e+09,4.620873e+09
...,...,...,...,...,...,...
59,4620734060,719,1005.0,4.620734e+09,4.620734e+09,NaN
60,4620733010,714,1006.0,4.620733e+09,4.620733e+09,NaN
61,4620740040,725,1007.0,4.620740e+09,4.620740e+09,NaN
62,4620733020,715,1006.0,4.620733e+09,4.620733e+09,NaN


In [754]:
hh

,nolog,facmen,nbveh,nbper,smlog,sdrlog,ct_2011,xmtmlog,ymtmlog,cma,hh_type,hid
0,1000002,29.52,1,2,622,2464008,4.620700e+09,293467.0,5062928.0,462,2,0
1,1000013,16.18,2,3,110,2466023,4.620196e+09,299590.0,5048131.0,462,2,1
2,1000014,22.34,2,7,117,2466023,4.620600e+09,299421.0,5049501.0,462,2,2
3,1000015,13.16,2,4,103,2466023,4.620075e+09,299469.0,5037458.0,462,2,3
4,1000016,38.26,0,1,406,2465005,4.620635e+09,290831.0,5047206.0,462,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...
78726,9033510,25.78,1,2,107,2466023,4.620226e+09,293941.0,5044832.0,462,2,69724
78727,9033848,27.12,1,1,307,2458227,4.620857e+09,311427.0,5040014.0,462,2,69725
78728,9034248,34.70,0,2,117,2466023,4.620602e+09,298204.0,5048340.0,462,2,69726
78729,9034353,40.27,1,1,105,2466023,4.620127e+09,295284.0,5039598.0,462,2,69727


In [756]:
hh.merge(reg_sample[['sample_geo', 'CTUID_2']].drop_duplicates(), left_on='ct_2011', right_on='CTUID_2')

,nolog,facmen,nbveh,nbper,smlog,sdrlog,ct_2011,xmtmlog,ymtmlog,cma,hh_type,hid,sample_geo,CTUID_2
0,1000002,29.52,1,2,622,2464008,4.620700e+09,293467.0,5062928.0,462,2,0,663.0,4.620700e+09
1,1008141,32.41,1,1,622,2464008,4.620700e+09,293549.0,5062472.0,462,2,2083,663.0,4.620700e+09
2,1009194,17.92,2,2,622,2464008,4.620700e+09,293660.0,5062791.0,462,2,2383,663.0,4.620700e+09
3,1015515,17.71,1,2,622,2464008,4.620700e+09,293799.0,5062765.0,462,2,3913,663.0,4.620700e+09
4,1035780,32.41,1,1,622,2464008,4.620700e+09,293422.0,5062773.0,462,2,8946,663.0,4.620700e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69684,1289250,18.90,1,7,103,2466023,4.620068e+09,299073.0,5038905.0,462,1,65570,70.0,4.620068e+09
69685,1211297,25.46,1,1,103,2466023,4.620091e+09,296486.0,5034122.0,462,2,51649,94.0,4.620091e+09
69686,9003694,29.94,4,3,633,2472032,4.620732e+09,259911.0,5037292.0,462,2,68118,1002.0,4.620732e+09
69687,9006407,44.98,1,1,633,2472032,4.620732e+09,261487.0,5038483.0,462,2,68620,1002.0,4.620732e+09


In [760]:
hh[~hh.ct_2011.isin(reg_sample.CTUID_2)].ct_2011.unique().astype(int)

array([4620832000, 4620189000, 4620229000, 4620145000])

In [549]:
hh = hh.merge(mapping_da_ct[['CTUID', 'new_ct']].drop_duplicates(), left_on="ct_2011", right_on="CTUID", how='left')

In [550]:
hh.loc[hh.new_ct.isna()].ct_2011.unique()

array([4.62073300e+09, 4.62075005e+09, 4.62073401e+09, 4.62068506e+09,
       4.62082501e+09, 4.62072508e+09, 4.62074002e+09, 4.62065003e+09,
       4.62083200e+09, 4.62007000e+09, 4.62090307e+09, 4.62006900e+09,
       4.62018900e+09, 4.62022900e+09, 4.62014500e+09, 4.62073202e+09])

In [551]:
unmatched_ct_2011_2016 = hh.loc[hh.new_ct.isna()].ct_2011.unique()  # 2013 HTS Census Tracts with no match from the Census Tract 2016 version

In [552]:
start = int(max(hh.new_ct) + 1)
end = start + len(hh.loc[hh.new_ct.isna()].ct_2011.unique())

In [553]:
ct_2011 = dict(zip(hh.loc[hh.new_ct.isna(), 'ct_2011'].drop_duplicates(), range(start, end)))

In [554]:
hh.loc[hh.new_ct.isna(), 'new_ct'] = [ct_2011.get(x, None) for x in hh.loc[hh.new_ct.isna(), 'ct_2011']]

In [555]:
mapping_ct_2011_2016 = pd.read_csv("../data/shp/mapping_ct_2011_ct_2016.csv")
mapping_ct_2011_2016 *= 1000

mapping_ct_2011_2016 = mapping_ct_2011_2016[mapping_ct_2011_2016.CTUID != mapping_ct_2011_2016.CTUID_2]

mapping_ct_2011_2016 = mapping_ct_2011_2016[["CTUID", "CTUID_2"]]

In [556]:
ct_2011_2016 = hh[['ct_2011', 'CTUID', 'new_ct']].drop_duplicates().rename(columns={'new_ct': 'new_ct_2011'})

In [557]:
ct_2011_2016.loc[~ct_2011_2016.CTUID.isna(), 'new_ct_2016'] = ct_2011_2016['new_ct_2011']

In [558]:
ct_2011_2016

,ct_2011,CTUID,new_ct_2011,new_ct_2016
0,4.620700e+09,4.620700e+09,663.0,663.0
1,4.620196e+09,4.620196e+09,212.0,212.0
2,4.620600e+09,4.620600e+09,490.0,490.0
3,4.620075e+09,4.620075e+09,78.0,78.0
4,4.620635e+09,4.620635e+09,543.0,543.0
...,...,...,...,...
51649,4.620091e+09,4.620091e+09,94.0,94.0
51803,4.620229e+09,NaN,939.0,NaN
67768,4.620145e+09,NaN,940.0,NaN
68118,4.620732e+09,NaN,941.0,NaN


In [559]:
test = mapping_ct_2011_2016[mapping_ct_2011_2016.CTUID_2.isin(unmatched_ct_2011_2016)].astype(int).rename(columns={'CTUID_2': 'ct_2011'})

In [560]:
test = test.merge(mapping_da_ct[['CTUID', 'new_ct']].drop_duplicates(), on='CTUID')
test.rename(columns={'new_ct': 'new_ct_2016'}, inplace=True)

In [561]:
test = test.merge(ct_2011_2016, on='ct_2011')

In [562]:
test

,CTUID_x,ct_2011,new_ct_2016_x,CTUID_y,new_ct_2011,new_ct_2016_y
0,4620903130,4620903070,906,NaN,936.0,NaN
1,4620903140,4620903070,907,NaN,936.0,NaN
2,4620734060,4620734010,719,NaN,928.0,NaN
3,4620734070,4620734010,720,NaN,928.0,NaN
4,4620650050,4620650030,572,NaN,933.0,NaN
5,4620650040,4620650030,571,NaN,933.0,NaN
6,4620725090,4620725080,698,NaN,931.0,NaN
7,4620725100,4620725080,699,NaN,931.0,NaN
8,4620685080,4620685060,639,NaN,929.0,NaN
9,4620685070,4620685060,638,NaN,929.0,NaN


In [563]:
new_ct_2011_2016 = dict(zip(test.new_ct_2011, test.new_ct_2016_x))

In [564]:
new_ct_2011_2016

{936.0: 907,
 928.0: 720,
 933.0: 571,
 931.0: 699,
 929.0: 638,
 927.0: 730,
 930.0: 779,
 937.0: 71,
 926.0: 714,
 932.0: 725,
 935.0: 74}

In [565]:
ct_2011_2016.loc[ct_2011_2016.new_ct_2016.isna(), 'new_ct_2016'] = [new_ct_2011_2016.get(x, None) for x in ct_2011_2016.loc[ct_2011_2016.new_ct_2016.isna()].new_ct_2011]

In [504]:
reg_sample = ct_2011_2016[~ct_2011_2016.new_ct_2016.isna()][['new_ct_2011', 'new_ct_2016']].astype(int)

In [537]:
reg_sample

,new_ct_2011,new_ct_2016,diff
0,663,663,0
1,212,212,0
2,490,490,0
3,78,78,0
4,543,543,0
...,...,...,...
18079,138,138,0
22196,165,165,0
48829,70,70,0
51649,94,94,0


In [505]:
reg_sample.rename(columns={'new_ct_2011': 'sample_geo', 'new_ct_2016': 'region'}).to_csv("V_1/region_sample_mapping.csv", index=False)

In [511]:
hh.rename(columns={"new_ct":"sample_geo", 'nbper': 'hh_size'}, inplace=True)

In [512]:
hh.loc[hh.hh_size > 4, "hh_size"] = 5

In [525]:
hh[["hid", "sample_geo", "hh_size" ,"hh_type"]].astype(int).to_csv("V_1/household_sample.csv", index=False)

In [530]:
geo_sample[geo_sample.new_ct_2011 == 934]

,ct_2011,CTUID_x,new_ct_2011,new_ct_2016,DAUID,CTUID_y,region,geo


## Persons

### Age

+ 1 : 0 à 4 ans
+ 2 : 5 à 9 ans
+ 3 : 10 à 14 ans
+ 4 : 15 à 19 ans
+ 5 : 20 à 24 ans
+ 6 : 25 à 29 ans
+ 7 : 30 à 34 ans
+ 8 : 35 à 39 ans
+ 9 : 40 à 44 ans
+ 10 : 45 à 49 ans
+ 11 : 50 à 54 ans
+ 12 : 55 à 59 ans
+ 13 : 60 à 64 ans
+ 14 : 65 à 69 ans
+ 15 : 70 à 74 ans
+ 16 : 75 ans et +


### Age 

In [514]:
def compute_age_category(age):
    if age >= 80:
        result = 16
    else:
        result = age // 5 + 1
    return result

In [515]:
pp['age'] = pp['age'].apply(compute_age_category)

### Driving license 

In [516]:
pp['p_licence'] = pp['percond']
pp.loc[(pp['p_licence'] == 3)| (pp['p_licence'] == 4), 'p_licence'] = 2

### Occupation 

Not used for the V0

#### Workplace

Not used for the V0

In [517]:
pp

,id_personne,nolog,noper,facper11,age,sexe,percond,occper,mobil,p_licence
0,1000002_1,1000002,1,28.40,8,2,1,1,1,1
1,1000002_2,1000002,2,25.99,8,1,1,5,1,1
2,1000013_1,1000013,1,16.36,10,2,1,7,1,1
3,1000013_2,1000013,2,20.13,10,1,1,1,1,1
4,1000013_3,1000013,3,21.08,4,1,5,3,1,5
...,...,...,...,...,...,...,...,...,...,...
188740,9034248_1,9034248,1,35.37,7,2,2,7,1,2
188741,9034248_2,9034248,2,47.00,7,1,1,7,2,1
188742,9034353_1,9034353,1,44.24,8,1,1,5,2,1
188743,9057507_1,9057507,1,38.06,7,2,1,1,1,1


In [518]:
pp['hid'] = [hh_id_dict[x] for x in pp.nolog]

In [519]:
pp['pid'] = range(len(pp))

In [520]:
pp.rename(columns={"sexe" : "sex"}, inplace=True)

In [521]:
pp = pp.merge(hh[['hid', 'sample_geo']], on='hid')

In [522]:
pp

,id_personne,nolog,noper,facper11,age,sex,percond,occper,mobil,p_licence,hid,pid,sample_geo
0,1000002_1,1000002,1,28.40,8,2,1,1,1,1,0,0,663.0
1,1000002_2,1000002,2,25.99,8,1,1,5,1,1,0,1,663.0
2,1000013_1,1000013,1,16.36,10,2,1,7,1,1,1,2,212.0
3,1000013_2,1000013,2,20.13,10,1,1,1,1,1,1,3,212.0
4,1000013_3,1000013,3,21.08,4,1,5,3,1,5,1,4,212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
167863,9034248_1,9034248,1,35.37,7,2,2,7,1,2,69726,167863,494.0
167864,9034248_2,9034248,2,47.00,7,1,1,7,2,1,69726,167864,494.0
167865,9034353_1,9034353,1,44.24,8,1,1,5,2,1,69727,167865,137.0
167866,9057507_1,9057507,1,38.06,7,2,1,1,1,1,69728,167866,790.0


In [524]:
pp[["hid", "pid", "sample_geo", "sex", "age", "p_licence"]].astype(int).to_csv("V_1/person_sample.csv", index=False)

## Geographic level mapping 

### Regional <-> GEO

In [443]:
geo_reg = pd.read_csv("V_1/matching_da_ct.csv")

In [444]:
geo_reg.rename(columns={'new_ct':'region', 'new_da':'geo'}, inplace=True)

In [464]:
geo_reg[["region", "geo"]].to_csv("V_1/region_geo_mapping.csv", index=False)

### Geo <-> Sample 

In [450]:
geo_sample = ct_2011_2016.merge(geo_reg, left_on='new_ct_2016', right_on='region')

In [451]:
geo_sample

,ct_2011,CTUID_x,new_ct_2011,new_ct_2016,DAUID,CTUID_y,region,geo
0,4.620700e+09,4.620700e+09,663.0,663,24640267,4620700030,663,1585
1,4.620700e+09,4.620700e+09,663.0,663,24640268,4620700030,663,1586
2,4.620700e+09,4.620700e+09,663.0,663,24640245,4620700030,663,1563
3,4.620700e+09,4.620700e+09,663.0,663,24640246,4620700030,663,1564
4,4.620700e+09,4.620700e+09,663.0,663,24640247,4620700030,663,1565
...,...,...,...,...,...,...,...,...
6177,4.620068e+09,4.620068e+09,70.0,70,24661111,4620068000,70,3272
6178,4.620068e+09,4.620068e+09,70.0,70,24661113,4620068000,70,3273
6179,4.620068e+09,4.620068e+09,70.0,70,24663434,4620068000,70,5438
6180,4.620091e+09,4.620091e+09,94.0,94,24663382,4620091000,94,5388


In [463]:
geo_sample[['new_ct_2011', 'geo']].astype(int).rename(columns={'new_ct_2011': 'sample_geo'}).to_csv("V_1/geo_sample_mapping.csv", index=False)